Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Continue to clean and explore your data.
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint.

In [1]:
import pandas as pd
# importing/instantiating all the csv's in case I want to use any of the other data as well
# main DF
df = pd.read_csv('C:/users/Stewa/Documents/Downloads/steam-store-games/steam.csv')
# More csvs in the dataset for extra data
tag_data = pd.read_csv('C:/users/Stewa/Documents/Downloads/steam-store-games/steamspy_tag_data.csv')
media = pd.read_csv('C:/users/Stewa/Documents/Downloads/steam-store-games/steam_media_data.csv')
descrip = pd.read_csv('C:/users/Stewa/Documents/Downloads/steam-store-games/steam_description_data.csv')

In [2]:
# check for nulls/clean them up
df.isnull().sum()
# instantiate a df_clean so i can have a version to work with, and drop NaNs to start
df_clean = df.dropna()
# check it out now
df_clean.shape
df_clean.isnull().sum()

appid               0
name                0
release_date        0
english             0
developer           0
publisher           0
platforms           0
required_age        0
categories          0
genres              0
steamspy_tags       0
achievements        0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
owners              0
price               0
dtype: int64

In [3]:
#defining my target to explore and validate that ts a good target, and feature pool
##########
potential_target = df_clean['price']
features_all = df_clean.drop(['price'], axis=1)
##########

In [4]:
# check out my target a bit, see what the common range is for prices
potential_target.describe()
# Average price is way lower than i thought

count    4519.000000
mean        8.210487
std         6.994366
min         0.000000
25%         3.990000
50%         6.990000
75%        10.990000
max        95.990000
Name: price, dtype: float64

In [5]:
print(potential_target.value_counts(normalize=True))
''' 
I gotta shorten this list, going to add a column for price range;
ie: $1-$5,$5-$10, etc.

My baseline is terrible, at only 17%, because of how many options 
there are, this is why i need to get that number down
'''

6.99     0.170170
3.99     0.126355
14.99    0.065280
0.00     0.062403
10.99    0.061297
           ...   
36.99    0.000221
2.29     0.000221
4.65     0.000221
95.99    0.000221
11.69    0.000221
Name: price, Length: 113, dtype: float64


' \nI gotta shorten this list, going to add a column for price range;\nie: $1-$5,$5-$10, etc.\n\nMy baseline is terrible, at only 17%, because of how many options \nthere are, this is why i need to get that number down\n'

In [6]:
(df_clean['price'] < 5) & (df_clean['price'] >= 1)

0       False
1        True
2        True
3        True
4        True
        ...  
4514     True
4515     True
4516     True
4517     True
4518     True
Name: price, Length: 4519, dtype: bool

In [14]:
def label_race (row):
    if (row['price'] >= .50) & (row['price'] <= 5) :
      return '1-5'
    if (row['price'] > 5) & (row['price'] <= 9.99) :
      return '5-10'
    if (row['price'] >= 10) & (row['price'] <= 14.99) :
      return '10-15'
    if (row['price'] >= 15) & (row['price'] <= 24.99) :
      return '15-25'
    if (row['price'] >= 25) & (row['price'] <= 70) :
      return '25+'
    return 'outlier'

df_clean['price_range'] = df_clean.apply(lambda row: label_race(row), axis=1)

C:\Users\stewa\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
df_clean.head()
# drop all my outliers
df_clean = df_clean[~df_clean.price_range.str.contains("outlier")]

In [16]:

''' 
This is a much better baseline to work with, and price format
in general. further down the line I'll look into how to predict 
specific prices, but for exploration/tinkering purposes, this will do.
This also turns this into a classification problem, as the options are 
3 different strings, not a continuous number
'''
df_clean['price_range'].value_counts(normalize=True)

5-10     0.359235
1-5      0.350024
10-15    0.184459
15-25    0.077232
25+      0.029051
Name: price_range, dtype: float64

In [17]:
# Define it as my target
target = df_clean['price_range']



In [18]:
'''
Now we know this is a classfication problem, and that my baseline 
by guessing would be 35%, shouldn't be too hard to beat!

Next, lets throw together a quick model using mostly old code,
everything as features, and just see how close it gets
'''

"\nNow we know this is a classfication problem, and that my baseline \nby guessing would be 35%, shouldn't be too hard to beat!\n\nNext, lets throw together a quick model using mostly old code,\neverything as features, and just see how close it gets\n"

In [19]:
#lets split the data into train,val,test!
# first, I need to cast release_date to dt format
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], infer_datetime_format=True)



train = df_clean.loc[(df_clean['release_date'] <= pd.datetime(2014,12,31)) 
                     & (df_clean['release_date'] >= pd.datetime(1997,1,1))]

test = df_clean.loc[(df_clean['release_date'] >= pd.datetime(2015,1,1))
                   & (df_clean['release_date'] <= pd.datetime(2019,12,31))]




In [20]:
df_clean.shape, train['release_date'].shape, test['release_date'].shape

((4234, 19), (3172,), (1062,))

In [21]:
train.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,price_range
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0.0,124534.0,3339.0,17612.0,317.0,10000000-20000000,7.19,5-10
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0.0,3318.0,633.0,277.0,62.0,5000000-10000000,3.99,1-5
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0.0,3416.0,398.0,187.0,34.0,5000000-10000000,3.99,1-5
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0.0,1273.0,267.0,258.0,184.0,5000000-10000000,3.99,1-5
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0.0,5250.0,288.0,624.0,415.0,5000000-10000000,3.99,1-5


In [22]:
# instantiate target and features
target = 'price_range'
features = ['english', 'developer', 'publisher', 'platforms', 'required_age',
            'categories', 'genres', 'steamspy_tags', 'achievements', 'positive_ratings',
            'negative_ratings',	'average_playtime', 'median_playtime', 'owners']
# lets create my matrices and vectors
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [23]:
# import everything for randomforest/imputer/encoders
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# Im going to use random forest, and ordinal encoding
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = {
    'randomforestclassifier__n_estimators': range(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0.2, 0.8), 
}
# going for a total of 30 different tests, so i can be a bit more accurate,
#but not take the entire sprint time to compute
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    random_state=42
)

search.fit(X_train, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:   18.5s remaining:   10.6s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:   23.8s remaining:    7.2s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:   26.4s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   31.1s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [24]:
print('Validation accuracy', search.best_score_)
# improved it a little, but ~42% accuracy is still not even clost to good.

# But more to come in future days! I'm having fun messing around with my own dataset

Validation accuracy 0.4271752837326608
